In [5]:
import traceback

try:
    adfl;jalfdkjad
except Exception as e:
    error_traceback = traceback.format_exc()
    print(f"Error: {e}\nTraceback: {error_traceback}")

Error: name 'adfl' is not defined
Traceback: Traceback (most recent call last):
  File "/var/folders/xz/gzjrrx4d4jd2v3ptdyp9kpc80000gn/T/ipykernel_82646/2111086033.py", line 4, in <module>
    adfl;jalfdkjad
    ^^^^
NameError: name 'adfl' is not defined

